In [2]:


from math import atan, pi, radians, cos, sin
import numpy as np
from numpy.linalg import inv, norm
from numpy import array, asarray, matrix
from math import *
import matplotlib.pyplot as plt



# def point_to_rad(p1, p2): # converts 2D cartesian points to polar angles in range 0 - 2pi
        
#     if (p1 > 0 and p2 >= 0): return atan(p2/(p1))
#     elif (p1 == 0 and p2 >= 0): return pi/2
#     elif (p1 < 0 and p2 >= 0): return -abs(atan(p2/p1)) + pi
#     elif (p1 < 0 and p2 < 0): return atan(p2/p1) + pi
#     elif (p1 > 0 and p2 < 0): return -abs(atan(p2/p1)) + 2*pi
#     elif (p1 == 0 and p2 < 0): return pi * 3/2
#     elif (p1 == 0 and p2 == 0): return pi * 3/2 # edge case


from math import atan2, pi

def point_to_rad(p1, p2):
    # Converts 2D cartesian points to polar angles in range -pi to pi
    angle = atan2(p2, p1)
    
    # Adjust angle to be in the range 0 to 2pi
    if angle < 0:
        angle += 2*pi
    
    return angle
    
def RotMatrix3D(rotation=[0,0,0],is_radians=True, order='xyz'):
    
    roll, pitch, yaw = rotation[0], rotation[1], rotation[2]

    # convert to radians is the input is in degrees
    if not is_radians: 
        roll = radians(roll)
        pitch = radians(pitch)
        yaw = radians(yaw)
    
    # rotation matrix about each axis
    rotX = np.matrix([[1, 0, 0], [0, cos(roll), -sin(roll)], [0, sin(roll), cos(roll)]])
    rotY = np.matrix([[cos(pitch), 0, sin(pitch)], [0, 1, 0], [-sin(pitch), 0, cos(pitch)]])
    rotZ = np.matrix([[cos(yaw), -sin(yaw), 0], [sin(yaw), cos(yaw), 0], [0, 0, 1]])
    
    # rotation matrix order (default: pitch -> roll -> yaw)
    if order == 'xyz': rotationMatrix = rotZ * rotY * rotX
    elif order == 'xzy': rotationMatrix = rotY * rotZ * rotX
    elif order == 'yxz': rotationMatrix = rotZ * rotX * rotY
    elif order == 'yzx': rotationMatrix = rotX * rotZ * rotY
    elif order == 'zxy': rotationMatrix = rotY * rotX * rotZ
    elif order == 'zyx': rotationMatrix = rotX * rotY * rotZ
    
    return rotationMatrix # roll pitch and yaw rotation 


class kinematics():
    
    def __init__(self):
        
        # note: leg IDs
        left_front = 0
        left_back  = 1
        right_front= 2
        right_back = 3
        
        self.right_legs = [right_front, right_back]
        
        self.link_1 = 0.08
        self.link_2 = 0.213
        self.link_3 = 0.213
        self.phi = radians(90) 
        
        # body dimensions
        self.length = 0.3762
        self.width = 0.0935
        self.hight = 0.0
        
        # leg origins (FR, FL, RR, RL), i.e., the coordinate of j1
        self.leg_origins = np.matrix([[self.length/2, self.width/2, 0],
                          [-self.length/2, self.width/2, 0],
                          [-self.length/2, -self.width/2, 0],
                          [self.length/2, -self.width/2, 0],
                          [self.length/2, self.width/2, 0]])
        
    # this method adjust inputs to the IK calculator by adding rotation and 
    # offset of that rotation from the center of the robot
    def leg_IK(self, xyz, rot = [0,0,0], legID=0, is_radians=True, center_offset=[0,0,0]):

        #what is xyz? what does it represent? 

        
        # check is the leg is from the right side 
        is_right = (legID in self.right_legs)
        
        # add offset of each leg from the axis of rotation
        XYZ = asarray((inv(RotMatrix3D(rot,is_radians)) * \
            ((array(xyz) + self.leg_origins[legID,:] - array(center_offset)).transpose())).transpose())
        
        # subtract the offset between the leg and the center of rotation 
        # so that the resultant coordiante is relative to the origin (j1) of the leg
        xyz_ = asarray(XYZ - self.leg_origins[legID,:] + array(center_offset)).flatten()

        # calculate the angles and coordinates of the leg relative to the origin of the leg
        return self.leg_IK_calc(xyz_, is_right)


    # IK calculator
    def leg_IK_calc(self, xyz, is_right=False): 

        #Get the foot x, y, z coordinates
        x, y, z = xyz[0], xyz[1], xyz[2] 
        
        # length of vector projected on the YZ plane. equiv. to len_A = sqrt(y**2 + z**2)
        len_A = np.sqrt(y**2 + z**2)

        #Calculate angle between len_A and the leg's projection line on YZ plane
        a_2 = np.arcsin(np.sin(self.phi)*self.link_1/len_A)

        #Calculate angle between link_1 and length len_A
        a_3 = np.pi - a_2 - self.phi

        #Calculate the angle from the positive y-axis to the end-effector
        a_1 = point_to_rad(y,z)

        #Length of the leg project in the YZ plane
        # len_H = np.sqrt(len_A**2 -self.link_1**2) 

        # angle_1 = np.arctan2(len_A, y)
        # theta_1 = a_3 - angle_1

        #Angle of link1 about the x-axis in positive y-axis
        if is_right: 
            theta_1 = a_1 - a_3
        else: 
            theta_1 = a_1 + a_3

            if theta_1 >= 2*np.pi:
                theta_1 -= 2*np.pi

        
        #Hip joint coordinates
        j1 = array([0,0,0]) 
        
        #Thight joint coordinates 
        #x, y, z = j2[0], j2[1], j2[2]
        j2 = array([0,self.link_1*cos(theta_1),self.link_1*sin(theta_1)]) 

        #Foot joint coordinates
        j4 = array(xyz)
        #Vector from j2 to j4, i.e., the vector from the thight to the foot
        j4_2_vec = j4 - j2
    
        
        if is_right:
            R = theta_1 - self.phi - pi/2
        else: 
            R = theta_1 + self.phi - pi/2
        
        # create rotation matrix to work on a new 2D plane (XZ_)
        rot_mtx = RotMatrix3D([-R,0,0],is_radians=True)
        j4_2_vec_ = rot_mtx * (np.reshape(j4_2_vec,[3,1]))
        
        # xyz in the rotated coordinate system + offset due to link_1 removed
        x_, y_, z_ = j4_2_vec_[0], j4_2_vec_[1], j4_2_vec_[2]
        
        len_B = norm([x_, z_]) # norm(j4-j2)
        
        # handling mathematically invalid input, i.e., point too far away to reach
        if len_B >= (self.link_2 + self.link_3): 
            len_B = (self.link_2 + self.link_3) * 0.99999
            # self.node.get_logger().warn('target coordinate: [%f %f %f] too far away' % (x, y, z))
            print('target coordinate: [%f %f %f] too far away' % (x, y, z))
        
        # b_1 : angle between +ve x-axis and len_B (0 <= b_1 < 2pi)
        # b_2 : angle between len_B and link_2
        # b_3 : angle between link_2 and link_3
        b_1 = point_to_rad(x_, z_)  
        b_2 = acos((self.link_2**2 + len_B**2 - self.link_3**2) / (2 * self.link_2 * len_B)) 
        b_3 = acos((self.link_2**2 + self.link_3**2 - len_B**2) / (2 * self.link_2 * self.link_3))  
        
        # assuming theta_2 = 0 when the leg is pointing down (i.e., 270 degrees offset from the +ve x-axis)
        theta_2 = b_1 - b_2    
        theta_3 = pi - b_3
        
        # CALCULATE THE COORDINATES OF THE JOINTS FOR VISUALIZATION
        j1 = np.array([0,0,0])
        
        # calculate joint 3
        j3_ = np.reshape(np.array([self.link_2*cos(theta_2),0, self.link_2*sin(theta_2)]),[3,1])
        j3 = np.asarray(j2 + np.reshape(np.linalg.inv(rot_mtx)*j3_, [1,3])).flatten()
        
        # calculate joint 4
        j4_ = j3_ + np.reshape(np.array([self.link_3*cos(theta_2+theta_3),0, self.link_3*sin(theta_2+theta_3)]), [3,1])
        j4 = np.asarray(j2 + np.reshape(np.linalg.inv(rot_mtx)*j4_, [1,3])).flatten()
        
        # modify angles to match robot's configuration (i.e., adding offsets)
        angles = self.angle_corrector(angles=[theta_1, theta_2, theta_3], is_right=is_right)
        #angles = [theta_1, theta_2, theta_3]
        # print(degrees(angles[0]))
        return [angles[0], angles[1], angles[2], j1, j2, j3, j4]
    
    
    def base_pose(self, rot=[0,0,0], is_radians=True, center_offset=[0,0,0]):
        
        # offset due to non-centered axes of rotation
        offset = RotMatrix3D(rot, is_radians) * \
            (matrix(center_offset).transpose()) - matrix(center_offset).transpose()
        
        # rotate the base around the center of rotation (if there is no offset, then the center of 
        # rotation will be at the center of the robot)
        rotated_base = RotMatrix3D(rot, is_radians) * self.leg_origins.transpose() - offset
        return rotated_base.transpose()
       
    # get coordinates of leg joints relative to j1
    def leg_pose(self, xyz, rot, legID, is_radians, center_offset=[0,0,0]):
        
        # get the coordinates of each joints relative to the leg's origin
        pose_relative = self.leg_IK(xyz, rot, legID, is_radians, center_offset)[3:]
        
        # adjust the coordinates according to the robot's orientation (roll, pitch, yaw)
        pose_true = RotMatrix3D(rot,is_radians) * (array(pose_relative).transpose())
        return pose_true.transpose()
    
    # plot rectangular base where each corner represents the origin of leg
    def plot_base(self, ax, rot=[0,0,0], is_radians=True, center_offset=[0,0,0]):
        # get coordinates
        p = (self.base_pose(rot, is_radians, center_offset)).transpose()     
        # plot coordinates
        ax.plot3D(asarray(p[0,:]).flatten(), asarray(p[1,:]).flatten(), asarray(p[2,:]).flatten(), 'r')
        return
       
    # plot leg 
    def plot_leg(self, ax, xyz, rot=[0,0,0], legID=None, is_radians=True, center_offset=[0,0,0]):
        # get coordinates
        p = ((self.leg_pose(xyz, rot, legID, is_radians, center_offset) \
                + self.base_pose(rot,is_radians,center_offset)[legID]).transpose())
        # plot coordinates
        ax.plot3D(asarray(p[0,:]).flatten(), asarray(p[1,:]).flatten(), asarray(p[2,:]).flatten(), 'b')
        return

    def plot_robot(self, xyz, rot=[0,0,0], leg_N=4, is_radians=True, limit=0.250, center_offset=[0,0,0]):
    
        ax = self.ax_view(limit)  # set the view
        self.plot_base(ax,rot, is_radians, center_offset)  # plot base

        # plot legs
        for leg in range(leg_N):
            self.plot_leg(ax,xyz[leg],rot,leg, is_radians, center_offset) 
        
        # show figure
        plt.show()
        return
    

    # TO-DO : modify this function depending on your robot's configuration
    # adjusting angle for specific configurations of motors, incl. orientation
    # this will vary for each robot (possibly for each leg as well)
    def angle_corrector(self, angles=[0,0,0], is_right=True):
        angles[1] -= 1.5*pi; # add offset # 90 degrees
        if is_right:
            theta_1 = angles[0] - pi
            theta_2 = angles[1] + 0*pi/180 # 45 degrees initial offset #
        else: 
            if angles[0] > pi:  
                theta_1 = angles[0] - 2*pi
            else: theta_1 = angles[0]
            
            theta_2 = -angles[1] - 0*pi/180
        
        theta_3 = -angles[2] + 0*pi/180
        return [theta_1, theta_2, theta_3]
        
    # set view  
    @staticmethod
    def ax_view(limit):
        ax = plt.axes(projection="3d")
        ax.set_xlim(-limit, limit)
        ax.set_ylim(-limit, limit)
        ax.set_zlim(-limit, limit)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        return ax

In [47]:
# %matplotlib notebook
import matplotlib.pyplot as plt
from ipywidgets import *


# make the plot larger
plt.rcParams['figure.dpi'] = 150

k = kinematics()

# Define the update function
def update(roll, pitch, yaw, x, y, z, rot_x, rot_y, rot_z):
    # Define the foot positions
    #FR, FL, RR, RL
    foot_positions = [[x, y+0.08, z], [x, y+0.08, z], [x, y-0.08, z], [x, y-0.08, z]]

    
    # Plot the robot
    k.plot_robot(xyz=foot_positions, rot=[roll, pitch, yaw], 
                 is_radians=False, center_offset=[rot_x, rot_y, rot_z])
    
    # Calculate and print joint angles for each leg
    for i, foot_position in enumerate(foot_positions):
        angles = k.leg_IK(foot_position, rot=[roll, pitch, yaw], legID=i, is_radians=False, center_offset=[rot_x, rot_y, rot_z])
        print(f"Leg {i} angles: Hip: {angles[0]:.2f}, Thigh: {angles[1]:.2f}, Calf: {angles[2]:.2f}")
        #print angles in degree as well 
        #print(f"Leg {i} angles: Hip: {angles[0]*180/3.14159:.2f}, Thigh: {angles[1]*180/3.14159:.2f}, Calf: {angles[2]*180/3.14159:.2f}")

# Use interact to create sliders for adjusting the parameters
interact(update,
         roll=widgets.FloatSlider(value=0, min=-20, max=20, step=1),
         pitch=widgets.FloatSlider(value=0, min=-20, max=20, step=1),
         yaw=widgets.FloatSlider(value=0, min=-20, max=20, step=1),
         x=widgets.FloatSlider(value=0., min=-0.10, max=0.10, step=0.01),
         y=widgets.FloatSlider(value=0., min=-0.10, max=0.10, step=0.01),
         z=widgets.FloatSlider(value=-0.35, min=-0.4, max=-0.1, step=0.01),
         rot_x=widgets.FloatSlider(value=0, min=-0.25205/2, max=0.25205/2, step=0.01),
         rot_y=widgets.FloatSlider(value=0, min=-0.105577/2, max=0.105577/2, step=0.01),
         rot_z=widgets.FloatSlider(value=0, min=-0.25, max=0.25, step=0.01));



#Hip joint: -60 ~ 60 degrees, in radians: -1.0472 ~ 1.0472
#Thigh joint: -38 ~ 170 degrees, in radians: -0.663225115 ~ 2.96705973
#Calf joint: -156 ~ -48 degrees, in radians: -2.70526034 ~ -0.837758041
#FR: hip: 0 thigh: 1, calf: -2
#FL: hip: 0 thigh: 1, calf: -2
#RR: hip: 0 thigh: 1, calf: -2
#RL: hip: 0 thigh: 1, calf: -2

interactive(children=(FloatSlider(value=0.0, description='roll', max=20.0, min=-20.0, step=1.0), FloatSlider(v…

In [52]:

# Example usage
kin = kinematics()

# Assuming we want to find the joint angles for each leg with x, y = 0 and varying z
foot_positions = [
    [0, 0.08, -0.35],  # FL
    [0, 0.08, -0.35],  # FR
    [0, -0.08, -0.35],  # RR
    [0, -0.08, -0.35],  # RL
]


for i, foot_position in enumerate(foot_positions):
    angles = kin.leg_IK(foot_position, legID=i)
    #print leg name and angles in radians
    foot_name = ['FL', 'RL', 'FR', 'RR']
    
    print(f"{foot_name[i]}: hip: {angles[0]:.5f}, thigh: {angles[1]:.5f}, calf: {angles[2]:.5f}")


    # print(f" \n Leg {i} angles: Hip: {angles[0] * 180 / pi}, Thigh: {angles[1] * 180 / pi}, Calf: {angles[2] * 180 / pi}")
    

FL: hip: 0.00000, thigh: 0.60659, calf: -1.21318
RL: hip: 0.00000, thigh: 0.60659, calf: -1.21318
FR: hip: -0.00000, thigh: -0.60659, calf: -1.21318
RR: hip: -0.00000, thigh: -0.60659, calf: -1.21318


In [3]:

# Example usage
kin = kinematics()

# Assuming we want to find the joint angles for each leg with x, y = 0 and varying z
foot_positions = [
    [0, 0.08, -0.35],  # FL
    # [0, 0.08, -0.35],  # FR
    # [0, -0.08, -0.35],  # RR
    # [0, -0.08, -0.35],  # RL
]


for i, foot_position in enumerate(foot_positions):
    angles = kin.leg_IK(foot_position, legID=i)
    #print leg name and angles in radians
    foot_name = ['FL', 'RL', 'FR', 'RR']
    
    print(f"{foot_name[0]}: hip: {angles[0]:.5f}, thigh: {angles[1]:.5f}, calf: {angles[2]:.5f}")


    # print(f" \n Leg {i} angles: Hip: {angles[0] * 180 / pi}, Thigh: {angles[1] * 180 / pi}, Calf: {angles[2] * 180 / pi}")
    

FL: hip: 0.00000, thigh: 0.60659, calf: -1.21318
